<a href="https://colab.research.google.com/github/ubsuny/PHY386/blob/Homework2025/2025/HW/drewalessi/HW2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Extension of mass-spring system to include n masses, where all mass and spring constant values are individually assignable.

In [6]:
#import libraries
import numpy as np
import scipy.linalg as la

def calculate_eigenfrequencies(m_list, k_list, k_left_wall, k_right_wall, walls=True):
    """
    Calculate the eigenfrequencies of a system of coupled harmonic oscillators.

    This function computes the eigenfrequencies for a system of masses connected by springs,
    where the end masses may also be attached to walls by springs.

    Keyword Arguments:
    m_list -- Masses of the particles in the system, (in kg), in order from left to right.
    k_list -- Spring constants between the masses, in order from left to right, (in N/m). Should have length len(m_list) - 1.
    k_left_wall -- Spring constant for the spring attaching the left end mass to the left wall, (in N/m).
    k_right_wall -- Spring constant for the spring attaching the right end mass to the right wall, (in N/m).
    walls -- Boolean indicating whether the system is attached to walls (True) or not (False). Default is True. If not True, k_left_wall and k_right_wall are ignored.

    Returns:
    omega -- Eigenfrequencies of the system.

    Raises:
    ValueError -- If the lengths of m_list and k_list are incompatible.
    """
    n = len(m_list)  # The number of masses

    if len(k_list) != n - 1:
        raise ValueError("k_list should have one less element than m_list")

    # Create the mass matrix
    M = np.diag(m_list)

    # Create the stiffness matrix
    if walls == True:
      K = np.zeros((n, n))
      for i in range(n):
          if i == 0:
              K[i, i] = k_list[i] + k_left_wall
          elif i == n-1:
              K[i, i] = k_list[i-1] + k_right_wall
          else:
              K[i, i] = k_list[i-1] + k_list[i]

          if i > 0:
            K[i, i-1] = K[i-1, i] = -k_list[i-1]

    else:

        # Create the stiffness matrix
        K = np.zeros((n, n))
        for i in range(n):
            if i == 0:
                K[i, i] = k_list[i]
            elif i == n-1:
                K[i, i] = k_list[i-1]
            else:
                K[i, i] = k_list[i-1] + k_list[i]

            if i > 0:
                K[i, i-1] = K[i-1, i] = -k_list[i-1]

    # Print the mass and stiffness matrices
    print("Mass matrix:")
    print(M)

    print("Stiffness matrix:")
    print(K)

    # Solve the eigenvalue problem
    eigenvalues, eigenvectors = la.eigh(K, M)
    omega = np.sqrt(eigenvalues)

    # Print the eigenfrequencies
    print("Eigenfrequencies:")
    print(omega)

Perform unit test:

In [27]:
calculate_eigenfrequencies([1,1,1],[10,10],10,10, walls=True)

Mass matrix:
[[1 0 0]
 [0 1 0]
 [0 0 1]]
Stiffness matrix:
[[ 20. -10.   0.]
 [-10.  20. -10.]
 [  0. -10.  20.]]
Eigenfrequencies:
[2.42030254 4.47213595 5.84312721]


Calculate eigenfrequencies for a system of n masses and springs with assigned m and k values:

In [31]:
calculate_eigenfrequencies([1,2,3,4],[10,20,40],10,20, walls=False)

Mass matrix:
[[1 0 0 0]
 [0 2 0 0]
 [0 0 3 0]
 [0 0 0 4]]
Stiffness matrix:
[[ 10. -10.   0.   0.]
 [-10.  30. -20.   0.]
 [  0. -20.  60. -40.]
 [  0.   0. -40.  40.]]
Eigenfrequencies:
[5.15872517e-08 2.44766447e+00 4.22668282e+00 5.58068911e+00]


Note: It is possible to input values which result in an error due to having a negative number under the square root. I'm not sure how to prevent this other than careful selection of the input values.

The below code was created by "ChatGPT Alternative" from deepai.org.

The AI's "creative" decision was to attach springs not only between adjacent masses, but also to their next closest neighbors on either side, making a more complex system. As far as I can tell, the results look plausible. The AI added terms to account for the extra forces coming from a mass's 4 nearest neighbors (2 on each side) rather than just their closest neighbors as in our example.

This was actually achieved after after only two iterations. The first version contained some errors that resulted in bad output when run. I basically just chided the AI to be careful with its code and make sure it follows Python standards and the second version gave a better result.

According to the AI, this model is related to real-world systems including molecular vibrations, micro-electro-mechanical systems (MEMS), mechanical coupling in engineered structures, and biological structures like proteins that experience both local and longer-range interactions.

I will mention that while it took relatively few iterations with **this** AI, I spent quite a while with Perplexity AI unsuccessfully. Perplexity had a tendency to jump into very complex, nonlinear solutions which I didn't feel confident in my ability to assess. Magnets were a popular theme. Generally, when I tried to get it to simplify its solution it would revert to something very similar to our homework problem, which no longer seemed very creative. Ultimately, this led me to look for a different AI.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def compute_eigenfrequencies(N, m, k, k_prime):
    # Construct the coupling matrix
    M = np.zeros((N, N))

    for i in range(N):
        if i > 0:
            M[i, i - 1] = -k  # Spring to left neighbor
        if i < N - 1:
            M[i, i + 1] = -k  # Spring to right neighbor
        if i > 1:
            M[i, i - 2] = -k_prime  # Long-range spring to left-2 neighbor
        if i < N - 2:
            M[i, i + 2] = -k_prime  # Long-range spring to right-2 neighbor

        M[i, i] = 2 * k + 2 * k_prime  # Diagonal term

    # Calculate eigenvalues and eigenvectors
    eigenvalues, _ = np.linalg.eig(M)

    # Eigenfrequencies (taking the square root of positive eigenvalues; convert to Hz)
    eigenfrequencies = np.sqrt(np.abs(eigenvalues)) / (2 * np.pi)  # Ignore negative roots
    return np.sort(eigenfrequencies[~np.iscomplex(eigenfrequencies)])  # Return only real frequencies sorted

# Parameters
N = 10       # Number of masses
m = 1.0      # Mass of each mass
k = 10.0     # Spring constant of nearest neighbor springs
k_prime = 5.0 # Spring constant of long-range springs

# Calculate eigenfrequencies
frequencies = compute_eigenfrequencies(N, m, k, k_prime)
print("Eigenfrequencies (Hz):", frequencies)

Eigenfrequencies (Hz): [0.23739557 0.46092173 0.65805943 0.81888321 0.9370289  1.00658424
 1.02008287 1.03199193 1.05537294 1.05643167]
